In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, GRU, Dense

# Download NLTK resources if not already downloaded
nltk.download('punkt')
nltk.download('stopwords')

# Load your dataset
file_path = r"C:\Users\Vidya\Desktop\vidya\text.csv"
df = pd.read_csv(file_path)

# Assuming you have a 'text' column for input and a 'label' column for output
X = df['text']
y = df['label']



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Vidya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Vidya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Text preprocessing function
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    
    # Tokenization
    tokens = word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Stemming
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(word) for word in tokens]
    
    return ' '.join(stemmed_tokens)

# Apply text preprocessing
X_preprocessed = X.apply(preprocess_text)



In [3]:
# Split the preprocessed data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

# Tokenize the preprocessed text data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)



In [4]:
# Convert preprocessed text to sequences and pad them
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

max_len = 100  # Max sequence length
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_len, padding='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_len, padding='post')

# Define the model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=16, input_shape=(max_len, )))
model.add(Bidirectional(GRU(units=64)))  # Bidirectional GRU layer
model.add(Dense(units=6, activation='softmax'))  # Output layer (adjust units for your task)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()



C:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:81: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 100, 16)        │        80,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 128)            │        31,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 112,262 (438.52 KB)

 Trainable params: 112,262 (438.52 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
# Train the model
history = model.fit(X_train_padded, y_train, epochs=10, batch_size=32, validation_data=(X_test_padded, y_test))

Epoch 1/10
10421/10421 ━━━━━━━━━━━━━━━━━━━━ 349s 34ms/step - accuracy: 0.8252 - loss: 0.4631 - val_accuracy: 0.9220 - val_loss: 0.1558
Epoch 2/10
10421/10421 ━━━━━━━━━━━━━━━━━━━━ 351s 34ms/step - accuracy: 0.9245 - loss: 0.1453 - val_accuracy: 0.9243 - val_loss: 0.1407
Epoch 3/10
10421/10421 ━━━━━━━━━━━━━━━━━━━━ 358s 34ms/step - accuracy: 0.9288 - loss: 0.1306 - val_accuracy: 0.9245 - val_loss: 0.1379
Epoch 4/10
10421/10421 ━━━━━━━━━━━━━━━━━━━━ 357s 34ms/step - accuracy: 0.9310 - loss: 0.1233 - val_accuracy: 0.9223 - val_loss: 0.1372
Epoch 5/10
10421/10421 ━━━━━━━━━━━━━━━━━━━━ 364s 35ms/step - accuracy: 0.9323 - loss: 0.1194 - val_accuracy: 0.9248 - val_loss: 0.1380
Epoch 6/10
10421/10421 ━━━━━━━━━━━━━━━━━━━━ 369s 35ms/step - accuracy: 0.9348 - loss: 0.1144 - val_accuracy: 0.9233 - val_loss: 0.1430
Epoch 7/10
10421/10421 ━━━━━━━━━━━━━━━━━━━━ 366s 35ms/step - accuracy: 0.9362 - loss: 0.1101 - val_accuracy: 0.9211 - val_loss: 0.1431
Epoch 8/10
10421/10421 ━━━━━━━━━━━━━━━━━━━━ 358s 34ms/s

In [9]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test_padded, y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

2606/2606 ━━━━━━━━━━━━━━━━━━━━ 23s 9ms/step - accuracy: 0.9156 - loss: 0.1639
Test Loss: 0.16270995140075684
Test Accuracy: 0.9153931140899658


In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder


In [6]:

random_forest = RandomForestClassifier(n_estimators=100, random_state=42)


In [8]:
random_forest.fit(X_train_padded, y_train)


RandomForestClassifier(random_state=42)

In [10]:
y_pred = random_forest.predict(X_test_padded)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy}")

print(classification_report(y_test, y_pred))

Test Accuracy: 0.43555816799021135
              precision    recall  f1-score   support

           0       0.47      0.52      0.49     24201
           1       0.43      0.71      0.54     28164
           2       0.21      0.07      0.11      6929
           3       0.50      0.16      0.24     11441
           4       0.36      0.12      0.18      9594
           5       0.17      0.06      0.09      3033

    accuracy                           0.44     83362
   macro avg       0.36      0.27      0.27     83362
weighted avg       0.42      0.44      0.39     83362

